# Polinominal regression, Ridge, Lasso

## Content



* Quadric trend
* Forecasting
* Bias-Variance Decomposition
* Ridge regression
* LAsso regression

<img src="Polinominal_regression.PNG">

## Quadric trend

In [1]:
from sympy import *

A = Matrix([[1,1,1],[1,2,4],[1,3,9],[1,4,16],[1,5,25],[1,6,36],[1,7,49],[1,8,64],[1,9,81],[1,10,100]])\

\begin{align*}
    X^{T}:
\end{align*}

In [2]:
display(A.transpose())

Matrix([
[1, 1, 1,  1,  1,  1,  1,  1,  1,   1],
[1, 2, 3,  4,  5,  6,  7,  8,  9,  10],
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]])

In [3]:
y = Matrix([[0.59],[0.72],[0.93],[1.27],[2.58],[1.74],[2.07],[2.42],[2.73],[2.95]])

\begin{align*}
    y^{T}:
\end{align*}

In [4]:
display(y.transpose())

Matrix([[0.59, 0.72, 0.93, 1.27, 2.58, 1.74, 2.07, 2.42, 2.73, 2.95]])

In [5]:
z = A.transpose()
d = (z*A)

\begin{align*}
(X^{T} \cdot X)
\end{align*}

In [6]:
display(d)

Matrix([
[ 10,   55,   385],
[ 55,  385,  3025],
[385, 3025, 25333]])

In [7]:
l = d.inv()

\begin{align*}
(X^{T} \cdot X)^{-1}
\end{align*}

In [8]:
display(l)

Matrix([
[ 83/60,   -21/40,  1/24],
[-21/40, 637/2640, -1/48],
[  1/24,    -1/48, 1/528]])

In [9]:
f = l*A.transpose()

\begin{align*}
(X^{T} \cdot X)^{-1} \cdot X^{T}
\end{align*}

In [10]:
display(f)

Matrix([
[   9/10,     1/2,  11/60,  -1/20,   -1/5,  -4/15,   -1/4,  -3/20,   1/30,    3/10],
[-67/220, -83/660,   1/88, 47/440, 53/330, 19/110, 63/440, 19/264, -9/220, -43/220],
[   1/44,   1/132, -1/264,  -1/88,  -1/66,  -1/66,  -1/88, -1/264,  1/132,    1/44]])

In [11]:
u = f*y

\begin{align*}
\vec{w} = \left(X^T X\right)^{-1} X^T \vec{y}
\end{align*}

In [12]:
display(u)

Matrix([
[              0.1135],
[   0.373189393939394],
[-0.00950757575757577]])

## Forecasting

### Trend forecasting

\begin{align*}
y_{11 tr} = \hat{w}_1 + \hat{w}_2 \cdot 11 + \hat{w}_3 \cdot 11 ^{2}
\end{align*}

In [13]:
y11tr = u[0]+u[1]*11+u[2]*11**2
print('y11tr = ', y11tr)

y11tr =  3.06816666666666


\begin{align*}
{\hat{\sigma}_{tr}}^2 = \frac{1}{7} \cdot \sum^{10}_{k=1}
({y_k - \hat{w}_{1} - \hat{w}_2 \cdot k - \hat{w}_3 \cdot k^2})^2
\end{align*}

In [14]:
def RSS(b0, b1, b2, y):
    RSS = 0
    RSSi = 0
    for i in range(1, len(y)+1):
        RSSi = (y[i-1]-b0-b1*i - b2*i*i)**2
        RSS += RSSi
    D = RSS/(len(y)-2)
    return D

In [15]:
b0 = u[0]
b1 = u[1]
b2 = u[2]
D = RSS(b0,b1,b2,y)
print('D = ', D)

D =  0.115620208333333


### Expert forecasting

In [16]:
y11 = 3.42
s11 = 0.07


### Joint forecasting

\begin{align*}
y_{for 11} = \frac{
    \frac{y_{11}}{s_{11}^2}+\frac{y_{11tr}}{ \hat{\sigma}_{tr}^2}
    }
    {
    \frac{1}{s_{11}^2}+\frac{1}{ \hat{\sigma}_{tr}^2}
    }
\end{align*}

In [17]:
yfor11 = ((y11)/(s11**2)+(y11tr)/(D))/((1)/(s11**2)+(1)/(D))
print("Joint forecasting: ", yfor11)

Joint forecasting:  3.40569548329551


https://www.kaggle.com/kashnitsky/topic-4-linear-models-part-1-ols

## 3. Bias-Variance Decomposition

Let's talk a little about the error properties of linear regression prediction (in fact, this discussion is valid for all machine learning algorithms). We just covered the following:

- true value of the target variable is the sum of a deterministic function $f\left(\textbf{x}\right)$ and random error $\epsilon$: $y = f\left(\textbf{x}\right) + \epsilon$;
- error is normally distributed with zero mean and some variance: $\epsilon \sim \mathcal{N}\left(0, \sigma^2\right)$;
- true value of the target variable is also normally distributed: $y \sim \mathcal{N}\left(f\left(\textbf{x}\right), \sigma^2\right)$;
- we try to approximate a deterministic but unknown function $f\left(\textbf{x}\right)$ using a linear function of the covariates $\widehat{f}\left(\textbf{x}\right)$, which, in turn, is a point estimate of the function $f$ in function space (specifically, the family of linear functions that we have limited our space to), i.e. a random variable that has mean and variance.

So, the error at the point $\textbf{x}$ decomposes as follows:

$$\Large \begin{array}{rcl} 
\text{Err}\left(\textbf{x}\right) &=& \mathbb{E}\left[\left(y - \widehat{f}\left(\textbf{x}\right)\right)^2\right] \\
&=& \mathbb{E}\left[y^2\right] + \mathbb{E}\left[\left(\widehat{f}\left(\textbf{x}\right)\right)^2\right] - 2\mathbb{E}\left[y\widehat{f}\left(\textbf{x}\right)\right] \\
&=& \mathbb{E}\left[y^2\right] + \mathbb{E}\left[\widehat{f}^2\right] - 2\mathbb{E}\left[y\widehat{f}\right] \\
\end{array}$$

For clarity, we will omit the notation of the argument of the functions. Let's consider each member separately. The first two are easily decomposed according to the formula $\text{Var}\left(z\right) = \mathbb{E}\left[z^2\right] - \mathbb{E}\left[z\right]^2$:

$$\Large \begin{array}{rcl} 
\mathbb{E}\left[y^2\right] &=& \text{Var}\left(y\right) + \mathbb{E}\left[y\right]^2 = \sigma^2 + f^2\\
\mathbb{E}\left[\widehat{f}^2\right] &=& \text{Var}\left(\widehat{f}\right) + \mathbb{E}\left[\widehat{f}\right]^2 \\
\end{array}$$

Note that:

$$\Large \begin{array}{rcl} 
\text{Var}\left(y\right) &=& \mathbb{E}\left[\left(y - \mathbb{E}\left[y\right]\right)^2\right] \\
&=& \mathbb{E}\left[\left(y - f\right)^2\right] \\
&=& \mathbb{E}\left[\left(f + \epsilon - f\right)^2\right] \\
&=& \mathbb{E}\left[\epsilon^2\right] = \sigma^2
\end{array}$$

$$\Large \mathbb{E}[y] = \mathbb{E}[f + \epsilon] = \mathbb{E}[f] + \mathbb{E}[\epsilon] = f$$

And finally, we get to the last term in the sum. Recall that the error and the target variable are independent of each other:

$$\Large \begin{array}{rcl} 
\mathbb{E}\left[y\widehat{f}\right] &=& \mathbb{E}\left[\left(f + \epsilon\right)\widehat{f}\right] \\
&=& \mathbb{E}\left[f\widehat{f}\right] + \mathbb{E}\left[\epsilon\widehat{f}\right] \\
&=& f\mathbb{E}\left[\widehat{f}\right] + \mathbb{E}\left[\epsilon\right] \mathbb{E}\left[\widehat{f}\right]  = f\mathbb{E}\left[\widehat{f}\right]
\end{array}$$

Finally, let's bring this all together:

$$\Large \begin{array}{rcl} 
\text{Err}\left(\textbf{x}\right) &=& \mathbb{E}\left[\left(y - \widehat{f}\left(\textbf{x}\right)\right)^2\right] \\
&=& \sigma^2 + f^2 + \text{Var}\left(\widehat{f}\right) + \mathbb{E}\left[\widehat{f}\right]^2 - 2f\mathbb{E}\left[\widehat{f}\right] \\
&=& \left(f - \mathbb{E}\left[\widehat{f}\right]\right)^2 + \text{Var}\left(\widehat{f}\right) + \sigma^2 \\
&=& \text{Bias}\left(\widehat{f}\right)^2 + \text{Var}\left(\widehat{f}\right) + \sigma^2
\end{array}$$

With that, we have reached our ultimate goal -- the last formula tells us that the forecast error of any model of type $y = f\left(\textbf{x}\right) + \epsilon$ is composed of:

- squared bias: $\text{Bias}\left(\widehat{f}\right)$ is the average error for all sets of data;
- variance: $\text{Var}\left(\widehat{f}\right)$ is error variability, or by how much error will vary if we train the model on different sets of data;
- irremovable error: $\sigma^2$.

While we cannot do anything with the $\sigma^2$ term, we can influence the first two. Ideally, we'd like to negate both of these terms (upper left square of the picture), but, in practice, it is often necessary to balance between the biased and unstable estimates (high variance).

<img src="https://habrastorage.org/files/281/108/1e9/2811081e9eda44d08f350be5a9deb564.png" width="480">

Generally, as the model becomes more computational (e.g. when the number of free parameters grows), the variance (dispersion) of the estimate also increases, but bias decreases. Due to the fact that the training set is memorized completely instead of generalizing, small changes lead to unexpected results (overfitting). On the other side, if the model is too weak, it will not be able to learn the pattern, resulting in learning something different that is offset with respect to the right solution.

<img src="https://habrastorage.org/webt/mp/jr/fo/mpjrfodzlknbpirv2rs8fxf8-rs.png" width="480">

The Gauss-Markov theorem asserts that the OLS estimator of parameters of the linear model is the best for the class of linear unbiased estimator. This means that if there exists any other unbiased model $g$, from the same class of linear models, we can be sure that $Var\left(\widehat{f}\right) \leq Var\left(g\right)$.

https://github.com/mephistopheies/dds/blob/master/lr_040117/ipy/lecture.ipynb

## L2 Regularization ##

<center>or ridge regression</center>

One of the symptoms of overfitting: function try to interpolate data (pass throught each point). Magnitude of weights is increasing with the degree of polynomial. What about to add some penalty term for big weights? So general case of regularization with penalty term is:
$$\large \mathcal{L}_{reg} \left(X, \vec{y}, \vec{w}\right) = \mathcal{L}\left(X, \vec{y}, \vec{w}\right) + \lambda R\left(\vec{w}\right)$$
where:
* $\large \lambda$ is regularization parameter

https://github.com/mephistopheies/dds/blob/master/lr_040117/ipy/lecture.ipynb

Lets penalize for large L2 norm of the weights vector:
$$\large R\left(\vec{w}\right) = \frac{1}{2} \left\| \vec{w} \right\|_2^2 = \frac{1}{2} \sum_{j=1}^m w_j^2 = \frac{1}{2} \vec{w}^T \vec{w}$$
then new objective will be:
$$\large \mathcal{L}\left(X, \vec{y}, \vec{w} \right) = \frac{1}{2} \left(\vec{y} - X \vec{w}\right)^T \left(\vec{y} - X \vec{w}\right) + \frac{\lambda}{2} \vec{w}^T \vec{w}$$
Lets find derivative wrt to $\large \vec{w}$:
$$\large \begin{array}{rcl}\Large \frac{\partial \mathcal{L}}{\partial \vec{w}} &=& \frac{\partial}{\partial \vec{w}} \left(\frac{1}{2} \left(\vec{y} - X \vec{w}\right)^T \left(\vec{y} - X \vec{w}\right) + \frac{\lambda}{2} \vec{w}^T \vec{w}\right) \\
&=& \frac{\partial}{\partial \vec{w}}\left( \frac{1}{2} \left( \vec{y}^T \vec{y} -2\vec{y}^T X \vec{w} + \vec{w}^T X^T X \vec{w}\right) + \frac{\lambda}{2} \vec{w}^T \vec{w} \right) \\
&=& -X^T \vec{y} + X^T X \vec{w} + \lambda \vec{w}
\end{array}$$
And final solution:
$$\large \begin{array}{rcl} \frac{\partial \mathcal{L}}{\partial \vec{w}} = 0 &\Leftrightarrow& -X^T \vec{y} + X^T X \vec{w} + \lambda \vec{w} = 0 \\
&\Leftrightarrow& X^T X \vec{w} + \lambda \vec{w} = X^T \vec{y} \\
&\Leftrightarrow& \left(X^T X + \lambda E\right) \vec{w} = X^T \vec{y} \\
&\Leftrightarrow& \vec{w} = \left(X^T X + \lambda E\right)^{-1} X^T \vec{y}
\end{array}$$

https://www.kaggle.com/kashnitsky/topic-4-linear-models-part-1-ols

This type of regression is called ridge regression. The ridge is the diagonal matrix that we add to the $\textbf{X}^\text{T} \textbf{X}$ matrix to ensure that we get a regular matrix as a result.

<img src="https://habrastorage.org/files/452/f36/c79/452f36c79d7843109993de219dde7cd5.png">

Such a solution reduces dispersion but becomes biased because the norm of the vector of parameters is also minimized, which makes the solution shift towards zero. On the figure below, the OLS solution is at the intersection of the white dotted lines. Blue dots represent different solutions of ridge regression. It can be seen that by increasing the regularization parameter $\lambda$, we shift the solution towards zero.


<img src="https://habrastorage.org/files/fe5/709/fe6/fe5709fe61fd4f3b8832a61f5fa05b9c.png">

https://github.com/mephistopheies/dds/blob/master/lr_040117/ipy/lecture.ipynb

## L1 regularization ##
<center>or LASSO (least absolute shrinkage and selection operator)</center>

Lets penalty for large L1 norm of the weights vector:
$$\large R\left(\vec{w}\right) = \left\| \vec{w} \right\|_1 = \sum_{j=1}^m \left| w_j \right|$$
then new objective will be:
$$\large \mathcal{L}\left(X, \vec{y}, \vec{w} \right) = \frac{1}{2n} \sum_{i=1}^n \left(\vec{x_i}^T \vec{w} - y_i\right)^2 + \lambda \sum_{j=1}^m \left| w_j \right|$$
Unfortunately, in general, there is no explicit formula for optimal weights. We will use gradient descent to estimate optimal values of weights. Lets find gradient wrt weights:
$$\large \frac{\partial \mathcal{L}}{\partial w_j} = \frac{1}{n}\sum_{i=1}^n \left(\vec{x_i}^T \vec{w} - y_i\right) \vec{x_i} + \lambda \text{sign}(\vec{w})$$
So update formula of weights is:
$$\large \vec{w}_{\text{new}} := \vec{w} - \alpha \frac{\partial \mathcal{L}}{\partial \vec{w}}$$
where:
* $\large \alpha$ is step of gradient descent (learning rate)